In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/extended_stress_detection_data.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/extended_stress_detection_data.csv'

In [ ]:
df

# Stress Detection

## 1. Data Cleaning and Preprocessing


In [ ]:
# Import all necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, confusion_matrix, classification_report,
                             roc_curve, roc_auc_score)
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


In [ ]:
# Check basic information
print("Dataset Shape:", df.shape)
print("\n" + "="*50)
print("Column Names and Types:")
print(df.dtypes)
print("\n" + "="*50)
print("Missing Values:")
print(df.isnull().sum())
print("\n" + "="*50)
print("Basic Statistics:")
print(df.describe())


In [ ]:
# Check target variable distribution
print("Target Variable Distribution:")
print(df['Stress_Detection'].value_counts())
print("\nTarget Variable Percentage:")
print(df['Stress_Detection'].value_counts(normalize=True) * 100)


In [ ]:
# Handle missing values
print("Handling missing values...")
df_cleaned = df.copy()

# Fill numerical missing values with median
numerical_cols = df_cleaned.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    if df_cleaned[col].isnull().sum() > 0:
        df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)

# Fill categorical missing values with mode
categorical_cols = df_cleaned.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df_cleaned[col].isnull().sum() > 0:
        df_cleaned[col].fillna(df_cleaned[col].mode()[0], inplace=True)

print("Missing values after cleaning:")
print(df_cleaned.isnull().sum().sum())


In [ ]:
# Handle outliers using IQR method for key numerical features
def remove_outliers_iqr(df, column, multiplier=1.5):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Check for negative values in Social_Interactions (noticed in the data)
print("Negative values in Social_Interactions:", (df_cleaned['Social_Interactions'] < 0).sum())
df_cleaned = df_cleaned[df_cleaned['Social_Interactions'] >= 0]

print(f"\nDataset shape after cleaning: {df_cleaned.shape}")


## 2. Exploratory Data Analysis (EDA)


In [ ]:
# Visualize target variable distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
df_cleaned['Stress_Detection'].value_counts().plot(kind='bar', ax=axes[0], color=['#2ecc71', '#f39c12', '#e74c3c'])
axes[0].set_title('Stress Detection Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Stress Level')
axes[0].set_ylabel('Count')
axes[0].tick_params(rotation=0)

# Pie chart
df_cleaned['Stress_Detection'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%',
                                                     colors=['#2ecc71', '#f39c12', '#e74c3c'])
axes[1].set_title('Stress Detection Percentage', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()


In [ ]:
# Correlation heatmap for numerical features
numerical_features = df_cleaned.select_dtypes(include=[np.number]).columns.tolist()

plt.figure(figsize=(16, 12))
correlation_matrix = df_cleaned[numerical_features].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Numerical Features', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()


In [ ]:
# Categorical features analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
categorical_features = ['Gender', 'Marital_Status', 'Smoking_Habit', 'Meditation_Practice']

for idx, feature in enumerate(categorical_features):
    ax = axes[idx//2, idx%2]
    pd.crosstab(df_cleaned[feature], df_cleaned['Stress_Detection']).plot(
        kind='bar', ax=ax, color=['#2ecc71', '#f39c12', '#e74c3c']
    )
    ax.set_title(f'{feature} vs Stress Detection', fontsize=12, fontweight='bold')
    ax.set_xlabel(feature)
    ax.set_ylabel('Count')
    ax.legend(title='Stress Level')
    ax.tick_params(rotation=45)

plt.tight_layout()
plt.show()


## 3. Feature Engineering


In [ ]:
# Create a copy for feature engineering
df_fe = df_cleaned.copy()

# 1. Create new features
# Sleep efficiency (Sleep Quality / Sleep Duration)
df_fe['Sleep_Efficiency'] = df_fe['Sleep_Quality'] / (df_fe['Sleep_Duration'] + 1e-5)

# Work-Life balance indicator
df_fe['Work_Life_Balance'] = df_fe['Work_Hours'] / (df_fe['Physical_Activity'] + df_fe['Social_Interactions'] + 1)

# Health score (combination of health metrics)
df_fe['Health_Score'] = (df_fe['Blood_Pressure'] + df_fe['Cholesterol_Level'] + df_fe['Blood_Sugar_Level']) / 3

# Screen time to physical activity ratio
df_fe['Screen_Activity_Ratio'] = df_fe['Screen_Time'] / (df_fe['Physical_Activity'] + 1)

# Age groups
df_fe['Age_Group'] = pd.cut(df_fe['Age'], bins=[0, 30, 40, 50, 100], labels=['Young', 'Middle', 'Senior', 'Elderly'])

print("New features created!")
print(f"Total features now: {df_fe.shape[1]}")


In [ ]:
# Convert time features to numeric (extract hour)
def extract_hour(time_str):
    try:
        time_part = time_str.split()[0]
        hour = int(time_part.split(':')[0])
        if 'PM' in time_str and hour != 12:
            hour += 12
        elif 'AM' in time_str and hour == 12:
            hour = 0
        return hour
    except:
        return 0

df_fe['Wake_Up_Hour'] = df_fe['Wake_Up_Time'].apply(extract_hour)
df_fe['Bed_Time_Hour'] = df_fe['Bed_Time'].apply(extract_hour)

# Drop original time columns
df_fe = df_fe.drop(['Wake_Up_Time', 'Bed_Time'], axis=1)

print("Time features converted to numeric!")


In [ ]:
# Encode categorical variables
label_encoders = {}
categorical_columns = df_fe.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_columns.remove('Stress_Detection')  # Don't encode target yet

for col in categorical_columns:
    le = LabelEncoder()
    df_fe[col] = le.fit_transform(df_fe[col].astype(str))
    label_encoders[col] = le

print("Categorical features encoded!")
print(f"Encoded columns: {categorical_columns}")


In [ ]:
# Prepare features and target
X = df_fe.drop('Stress_Detection', axis=1)
y = df_fe['Stress_Detection']

# Encode target variable
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

print(f"Features shape: {X.shape}")
print(f"Target shape: {y_encoded.shape}")
print(f"\nTarget mapping: {dict(zip(le_target.classes_, le_target.transform(le_target.classes_)))}")


In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set size: {X_train_scaled.shape}")
print(f"Test set size: {X_test_scaled.shape}")


## 4. Model Training and Evaluation


In [ ]:
# Define multiple classification models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
}

print("Models initialized!")


In [ ]:
# Train and evaluate all models
results = []

for name, model in models.items():
    print(f"\nTraining {name}...")

    # Train the model
    model.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred = model.predict(X_test_scaled)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })

    print(f"{name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

# Create results dataframe
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('Accuracy', ascending=False)
print("\n" + "="*80)
print("MODEL PERFORMANCE SUMMARY")
print("="*80)
print(results_df.to_string(index=False))


In [ ]:
# Visualize model performance comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']

for idx, metric in enumerate(metrics):
    ax = axes[idx//2, idx%2]
    results_df.plot(x='Model', y=metric, kind='bar', ax=ax, color=colors[idx], legend=False)
    ax.set_title(f'{metric} Comparison', fontsize=14, fontweight='bold')
    ax.set_xlabel('Model', fontsize=12)
    ax.set_ylabel(metric, fontsize=12)
    ax.set_ylim([0, 1.1])
    ax.tick_params(axis='x', rotation=45)

    # Add value labels on bars
    for container in ax.containers:
        ax.bar_label(container, fmt='%.3f', fontsize=9)

plt.tight_layout()
plt.show()


In [ ]:
# Detailed evaluation of the best model
best_model_name = results_df.iloc[0]['Model']
best_model = models[best_model_name]

print(f"\n{'='*80}")
print(f"DETAILED EVALUATION OF BEST MODEL ON BASELINE MODEL BEFORE HYPERPARAMETER TUNING:\n {best_model_name}")
print(f"{'='*80}")

y_pred_best = best_model.predict(X_test_scaled)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best, target_names=le_target.classes_))


In [ ]:
# Confusion matrices for all models
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.ravel()

for idx, (name, model) in enumerate(models.items()):
    if idx < len(axes):
        y_pred = model.predict(X_test_scaled)
        cm = confusion_matrix(y_test, y_pred)

        sns.heatmap(cm, annot=True, fmt='d', cmap='YlOrRd', ax=axes[idx],
                   xticklabels=le_target.classes_, yticklabels=le_target.classes_,
                   cbar=False)
        axes[idx].set_title(f'{name}', fontsize=11, fontweight='bold')
        axes[idx].set_xlabel('Predicted')
        axes[idx].set_ylabel('Actual')

# Hide the last empty subplot
if len(models) < len(axes):
    fig.delaxes(axes[-1])

plt.suptitle('Confusion Matrices - All Models', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
# Feature importance (for tree-based models)
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False).head(15)

    plt.figure(figsize=(12, 8))
    sns.barplot(data=feature_importance, x='Importance', y='Feature', palette='viridis')
    plt.title(f'Top 15 Feature Importances - {best_model_name}', fontsize=16, fontweight='bold')
    plt.xlabel('Importance Score', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.tight_layout()
    plt.show()

    print("\nTop 10 Most Important Features:")
    print(feature_importance.head(10).to_string(index=False))
else:
    print(f"\nFeature importance not available for {best_model_name}")


In [ ]:
# Cross-validation scores for all models
print("\n" + "="*80)
print("CROSS-VALIDATION SCORES (5-Fold)")
print("="*80)

cv_results = []
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    cv_results.append({
        'Model': name,
        'Mean CV Score': cv_scores.mean(),
        'Std CV Score': cv_scores.std()
    })
    print(f"{name}: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

cv_results_df = pd.DataFrame(cv_results).sort_values('Mean CV Score', ascending=False)
print("\n" + cv_results_df.to_string(index=False))


In [ ]:
# Visualize cross-validation results
plt.figure(figsize=(14, 6))
x_pos = np.arange(len(cv_results_df))
plt.bar(x_pos, cv_results_df['Mean CV Score'], yerr=cv_results_df['Std CV Score'],
        color='steelblue', alpha=0.8, capsize=5, edgecolor='black')
plt.xlabel('Models', fontsize=12, fontweight='bold')
plt.ylabel('Cross-Validation Accuracy', fontsize=12, fontweight='bold')
plt.title('5-Fold Cross-Validation Scores for All Models', fontsize=14, fontweight='bold', pad=20)
plt.xticks(x_pos, cv_results_df['Model'], rotation=45, ha='right')
plt.ylim([0, 1.1])
plt.grid(axis='y', alpha=0.3)

# Add value labels
for i, v in enumerate(cv_results_df['Mean CV Score']):
    plt.text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()


## 5. Hyperparameter Tuning with GridSearchCV

In [ ]:
# Hyperparameter tuning with GridSearchCV and StratifiedKFold for KNN, Random Forest, and Logistic Regression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Define StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("="*80)
print(" "*20 + "🔧 HYPERPARAMETER TUNING")
print("="*80)

# 1) K-Nearest Neighbors (KNN) Tuning
print("\n1️⃣  Tuning K-Nearest Neighbors...")
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
    'metric': ['euclidean', 'manhattan']
}

grid_knn = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid_knn,
    scoring='accuracy',
    cv=cv,
    n_jobs=-1,
    verbose=0
)

grid_knn.fit(X_train_scaled, y_train)
y_pred_knn_tuned = grid_knn.best_estimator_.predict(X_test_scaled)
acc_knn_tuned = accuracy_score(y_test, y_pred_knn_tuned)

print(f"   Best params: {grid_knn.best_params_}")
print(f"   Best CV accuracy: {grid_knn.best_score_:.4f}")
print(f"   Test accuracy: {acc_knn_tuned:.4f}")

# 2) Random Forest Tuning
print("\n2️⃣  Tuning Random Forest...")
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt'],
    'class_weight': ['balanced']
}

grid_rf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid_rf,
    scoring='accuracy',
    cv=cv,
    n_jobs=-1,
    verbose=0
)

grid_rf.fit(X_train_scaled, y_train)
y_pred_rf_tuned = grid_rf.best_estimator_.predict(X_test_scaled)
acc_rf_tuned = accuracy_score(y_test, y_pred_rf_tuned)

print(f"   Best params: {grid_rf.best_params_}")
print(f"   Best CV accuracy: {grid_rf.best_score_:.4f}")
print(f"   Test accuracy: {acc_rf_tuned:.4f}")

# 3) Logistic Regression Tuning
print("\n3️⃣  Tuning Logistic Regression...")
param_grid_lr = {
    'penalty': ['l2'],
    'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
    'solver': ['lbfgs', 'saga'],
    'max_iter': [1000, 2000],
    'class_weight': ['balanced', None]
}

grid_lr = GridSearchCV(
    estimator=LogisticRegression(random_state=42),
    param_grid=param_grid_lr,
    scoring='accuracy',
    cv=cv,
    n_jobs=-1,
    verbose=0
)

grid_lr.fit(X_train_scaled, y_train)
y_pred_lr_tuned = grid_lr.best_estimator_.predict(X_test_scaled)
acc_lr_tuned = accuracy_score(y_test, y_pred_lr_tuned)

print(f"   Best params: {grid_lr.best_params_}")
print(f"   Best CV accuracy: {grid_lr.best_score_:.4f}")
print(f"   Test accuracy: {acc_lr_tuned:.4f}")

# Summary comparison
print("\n" + "="*80)
print(" "*15 + "📊 TUNED MODELS PERFORMANCE SUMMARY")
print("="*80)

tuned_results = pd.DataFrame({
    'Model': ['K-Nearest Neighbors', 'Random Forest', 'Logistic Regression'],
    'Best CV Accuracy': [grid_knn.best_score_, grid_rf.best_score_, grid_lr.best_score_],
    'Test Accuracy': [acc_knn_tuned, acc_rf_tuned, acc_lr_tuned]
})

print(tuned_results.to_string(index=False))
print("="*80)


## 6. Final Summary and Recommendations


In [ ]:
# Visualizations for tuned models
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1) Bar chart comparing CV vs Test accuracy
ax1 = axes[0]
x = np.arange(len(tuned_results))
width = 0.35

bars1 = ax1.bar(x - width/2, tuned_results['Best CV Accuracy'], width, label='CV Accuracy', color='steelblue', edgecolor='black')
bars2 = ax1.bar(x + width/2, tuned_results['Test Accuracy'], width, label='Test Accuracy', color='coral', edgecolor='black')

ax1.set_xlabel('Model', fontsize=12, fontweight='bold')
ax1.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax1.set_title('Tuned Models: CV vs Test Accuracy', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(tuned_results['Model'], rotation=15, ha='right')
ax1.legend()
ax1.set_ylim([0, 1.1])
ax1.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01, f'{height:.3f}', ha='center', va='bottom', fontsize=9)
for bar in bars2:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01, f'{height:.3f}', ha='center', va='bottom', fontsize=9)

# 2) Confusion matrices for tuned models
from sklearn.metrics import confusion_matrix

# Create subplots for confusion matrices
fig2, axes2 = plt.subplots(1, 3, figsize=(18, 5))

tuned_preds = [y_pred_knn_tuned, y_pred_rf_tuned, y_pred_lr_tuned]
tuned_names = ['KNN (Tuned)', 'Random Forest (Tuned)', 'Logistic Regression (Tuned)']

for idx, (name, y_pred) in enumerate(zip(tuned_names, tuned_preds)):
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes2[idx],
                xticklabels=le_target.classes_, yticklabels=le_target.classes_,
                cbar_kws={'label': 'Count'})
    axes2[idx].set_title(f'{name}', fontsize=12, fontweight='bold')
    axes2[idx].set_xlabel('Predicted')
    axes2[idx].set_ylabel('Actual')

plt.suptitle('Confusion Matrices - Tuned Models', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# 3) Classification reports for tuned models
print("\n" + "="*80)
print(" "*20 + "📋 DETAILED CLASSIFICATION REPORTS")
print("="*80)

for name, y_pred in zip(tuned_names, tuned_preds):
    print(f"\n{name}:")
    print("-" * 80)
    print(classification_report(y_test, y_pred, target_names=le_target.classes_))


In [ ]:
# Identify and display the BEST TUNED MODEL
print("\n" + "="*80)
print(" "*25 + "🏆 BEST MODEL SELECTION")
print("="*80)

# Sort by test accuracy to find the best model
tuned_results_sorted = tuned_results.sort_values('Test Accuracy', ascending=False)
best_tuned_model_name = tuned_results_sorted.iloc[0]['Model']
best_cv_acc = tuned_results_sorted.iloc[0]['Best CV Accuracy']
best_test_acc = tuned_results_sorted.iloc[0]['Test Accuracy']

print(f"\n🥇 WINNER: {best_tuned_model_name}")
print("="*80)
print(f"   ✓ Best CV Accuracy:   {best_cv_acc:.4f}")
print(f"   ✓ Test Accuracy:      {best_test_acc:.4f}")
print(f"   ✓ Improvement over CV: {(best_test_acc - best_cv_acc):.4f}")

# Get the best model parameters
if best_tuned_model_name == 'K-Nearest Neighbors':
    best_model = grid_knn.best_estimator_
    best_params = grid_knn.best_params_
    y_pred_best = y_pred_knn_tuned
elif best_tuned_model_name == 'Random Forest':
    best_model = grid_rf.best_estimator_
    best_params = grid_rf.best_params_
    y_pred_best = y_pred_rf_tuned
else:  # Logistic Regression
    best_model = grid_lr.best_estimator_
    best_params = grid_lr.best_params_
    y_pred_best = y_pred_lr_tuned

print(f"\n📋 Best Hyperparameters:")
print("-"*80)
for param, value in best_params.items():
    print(f"   • {param}: {value}")

# Detailed metrics for the best model
precision = precision_score(y_test, y_pred_best, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred_best, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred_best, average='weighted', zero_division=0)

print(f"\n📊 Comprehensive Metrics for {best_tuned_model_name}:")
print("-"*80)
print(f"   • Accuracy:  {best_test_acc:.4f}")
print(f"   • Precision: {precision:.4f}")
print(f"   • Recall:    {recall:.4f}")
print(f"   • F1-Score:  {f1:.4f}")
print("\n" + "="*80)

# Confusion matrix for the best model
print(f"\n📈 Confusion Matrix for Best Model: {best_tuned_model_name}")
cm = confusion_matrix(y_test, y_pred_best)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
            xticklabels=le_target.classes_, yticklabels=le_target.classes_,
            cbar_kws={'label': 'Count'}, linewidths=1, linecolor='black')
plt.title(f'🏆 Best Tuned Model: {best_tuned_model_name}\nTest Accuracy: {best_test_acc:.4f}',
          fontsize=14, fontweight='bold', pad=15)
plt.xlabel('Predicted', fontsize=12, fontweight='bold')
plt.ylabel('Actual', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

# Detailed classification report
print(f"\n📋 Detailed Classification Report for Best Model ({best_tuned_model_name}):")
print("="*80)
print(classification_report(y_test, y_pred_best, target_names=le_target.classes_))
print("="*80)